In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt 

tweets = pd.read_csv('combined.csv', encoding='utf-8')
tweets = tweets.drop(['tweet_date_created'], axis=1)
tweets = tweets.drop(['sentiment_score'], axis=1)

In [2]:
duplicates = tweets[tweets.duplicated(subset=['tweet_id'], keep=False)]

if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate tweet ids. Removing duplicates...")
    tweets.drop_duplicates(subset=['tweet_id'], inplace=True)
else:
    print("No duplicate tweet ids found.")

Found 762643 duplicate tweet ids. Removing duplicates...


In [3]:
tweets = tweets.drop(['language'], axis=1)
tweets = tweets.drop(['tweet_id'], axis=1)

In [4]:
# Define a dictionary that maps sentiment labels to numeric values
sentiment_map = {"NEUTRAL": 0, "POSITIVE": 1, "MIXED": 2, "NEGATIVE": 3}

# Map the sentiment labels to their numeric values
tweets['sentiment_values'] = tweets['sentiment'].map(sentiment_map)

In [5]:
tweets = tweets.drop(['sentiment'], axis=1)
tweets.head(5)

,tweet_text,sentiment_values
0,Bayer Leverkusen goalkeeper Bernd Leno will no...,0
1,Gary Speed v Blackburn at St James in 2001/02 ...,0
2,@ChelseaFC Don't make him regret it and start ...,0
3,"@LiverpoolFF @AnfieldEdition He's a liar, made...",3
4,@theesk @Everton Didn't realise Kenwright is d...,0


In [6]:
# Find the indices of rows with Mixed sentiment
mixed_indices = tweets[tweets['sentiment_values'] == 2].index

# Delete the rows with Mixed sentiment
tweets = tweets.drop(mixed_indices)

In [7]:
import pandas as pd

# Shuffle the DataFrame to ensure that the downsampling is random
tweets = tweets.sample(frac=1, random_state=42)

# Count the number of tweets in each sentiment class
counts = tweets['sentiment_values'].value_counts()

# Find the smallest class size
smallest_size = counts.min()

# Downsample each class to the smallest size
positive_tweets = tweets[tweets['sentiment_values'] == 1].sample(n=smallest_size, random_state=42)
negative_tweets = tweets[tweets['sentiment_values'] == 3].sample(n=smallest_size, random_state=42)
neutral_tweets = tweets[tweets['sentiment_values'] == 0].sample(n=smallest_size, random_state=42)

# Concatenate the downsampled DataFrames
tweets = pd.concat([positive_tweets, negative_tweets, neutral_tweets], ignore_index=True)

# Print the new counts of tweets in each class
print('No of positive tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 1])))
print('No of negative tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 3])))
print('No of neutral tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 0])))


No of positive tagged tweets is: 354501
No of negative tagged tweets is: 354501
No of neutral tagged tweets is: 354501


Customized Stopwords Function

In [9]:
import re
import string 
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from nltk.tokenize import word_tokenize

english_stopwords = stopwords.words('english')

def preprocess_text(text):
    # Get the default NLTK English stopwords
    
    # List of words to keep
    words_to_keep = {"off", "over", "under", "few", "more", "no", "not", "don't", "should", "should've", "aren't", 
                     "couldn't", "didn't", "doesn't", "hadn't", "haven't", "mustn't", "shouldn't", "wasn't", "weren't",
                     "won't", "wouldn't"}

    # Convert english_stopwords to a set, perform the subtraction, and convert the result back to a list
    custom_stopwords = list(set(english_stopwords) - words_to_keep)

    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    # Remove custom stopwords and join the words in a single string
    text = ' '.join([word for word in text.split() if word not in custom_stopwords])
    
    return text

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)


In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
import dill

# Split the dataset into input (X) and output (y)
X = tweets['processed_text'].values
y = tweets['sentiment_values'].values

# Split the dataset into 70% training and 30% combined validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary dataset (30% of the entire dataset) into 50% validation and 50% testing
# This results in 15% validation and 15% testing of the entire dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data size:", len(X_train))
print("Validation data size:", len(X_val))
print("Testing data size:", len(X_test))

Training data size: 744452
Validation data size: 159525
Testing data size: 159526


Customized Stopwords

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
import dill

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=None, norm='l1', use_idf=False)),
    ('clf', MultinomialNB(alpha=0.1, fit_prior=False))
])

text_clf.fit(X_train.tolist(), y_train)

y_pred = text_clf.predict(X_val.tolist())
accuracy = accuracy_score(y_val, y_pred)

print("Accuracy on test data: {:.4f}".format(accuracy))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_val, y_pred)
print("Classification Report:\n", cr)


Accuracy on test data: 0.7510
Confusion Matrix:
 [[31710  9759 11883]
 [ 5622 42865  4394]
 [ 4913  3146 45233]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.59      0.66     53352
           1       0.77      0.81      0.79     52881
           3       0.74      0.85      0.79     53292

    accuracy                           0.75    159525
   macro avg       0.75      0.75      0.75    159525
weighted avg       0.75      0.75      0.75    159525



No stopwords

In [8]:
import re
import string 
import pandas as pd
import nltk 
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

# Add custom stopwords
custom_stopwords = ['dont', 'shouldve', 'arent', 'couldnt', 'didnt', 'doesnt', 'hadnt', 'havent', 'mustnt', 'shouldnt', 'wasnt', 'werent', 
                    'wont', 'wouldnt']
english_stopwords.extend(custom_stopwords)

def preprocess_text(text):
    # Get the default NLTK English stopwords
    
    # List of words to keep
    #words_to_keep = {"off", "over", "under", "few", "more", "no", "not", "don't", "should", "should've", "aren't", 
    #                 "couldn't", "didn't", "doesn't", "hadn't", "haven't", "mustn't", "shouldn't", "wasn't", "weren't",
    #                 "won't", "wouldn't"}
    # Create a custom stopwords list
    #custom_stopwords = default_stopwords - words_to_keep
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    # Remove custom stopwords and join the words in a single string
    text = ' '.join([word for word in text.split() if word not in english_stopwords])
    return text


In [10]:
import pandas as pd
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)


In [11]:
# Filter the rows containing the word 'not' in the 'processed_text' column
rows_with_not = tweets.loc[tweets['processed_text'].str.contains(r'\bwont\b', regex=True)]
# Print the rows containing the word 'not'
print(rows_with_not)


Empty DataFrame
Columns: [tweet_text, sentiment_values, processed_text]
Index: []


In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
import dill

# Split the dataset into input (X) and output (y)
X = tweets['processed_text'].values
y = tweets['sentiment_values'].values

# Split the dataset into 70% training and 30% combined validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary dataset (30% of the entire dataset) into 50% validation and 50% testing
# This results in 15% validation and 15% testing of the entire dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data size:", len(X_train))
print("Validation data size:", len(X_val))
print("Testing data size:", len(X_test))

Training data size: 744452
Validation data size: 159525
Testing data size: 159526


# No stopwords Model Implementation

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
import dill

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1), max_features=None, norm='l1', use_idf=True)),
    ('clf', MultinomialNB(alpha=1.0, fit_prior=True))
])

text_clf.fit(X_train.tolist(), y_train)

y_pred = text_clf.predict(X_val.tolist())
accuracy = accuracy_score(y_val, y_pred)

print("Accuracy on test data: {:.4f}".format(accuracy))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_val, y_pred)
print("Classification Report:\n", cr)


Accuracy on test data: 0.7401
Confusion Matrix:
 [[30646 10398 12308]
 [ 6216 41547  5118]
 [ 4628  2786 45878]]
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.57      0.65     53352
           1       0.76      0.79      0.77     52881
           3       0.72      0.86      0.79     53292

    accuracy                           0.74    159525
   macro avg       0.74      0.74      0.74    159525
weighted avg       0.74      0.74      0.73    159525

